In [15]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import pathlib
import math
import difflib
from PIL import Image, ImageDraw, ImageFont


print(pathlib.Path().resolve())
pres_label_path = "Data/prescription/label/"
pres_image_path = "Data/prescription/image/"

dst = "Data/prescription/results/"
mapping_file = "Data/pill_categories/label-name.csv"

C:\Users\Admin\Desktop\Course\AIO2022\Team Hung An\Medicine_Pill_Image_Recognition_Challenge


In [ ]:
def write_text(image, result_text, point):
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # font_text = ImageFont.truetype("arial.ttf", 30)

    font_text = ImageFont.truetype("Data/Data-visualization/font/SVN-Arial-Regular.ttf", 20)

    draw =  ImageDraw.Draw(pil_img)
    point = (point[0], int(point[1]-20))
    draw.text(point, result_text, (255, 0, 0), font=font_text)
    cv2_img = cv2.cvtColor(np.asarray(pil_img), cv2.COLOR_RGB2BGR)
    return cv2_img


df = pd.read_csv(mapping_file)
cells = df['Drug name'].tolist()
AllDrugnames = []
for cell in cells:
    try:
        names = cell.split(" || ")
        for name in names:
            print("'"+name+"'")
            AllDrugnames.append(name)
    except:
        if math.isnan(cell):
            print(cell)


for index, pres_json_file in enumerate(os.listdir(pres_label_path)):
    pres_json_path = os.path.join(pres_label_path, pres_json_file)
    json_f = open(pres_json_path, mode='r', encoding='utf-8')
    pres_data = json.load(json_f)

    pres_img_path = os.path.join(pres_image_path, pres_json_file.replace("json", "png"))
    pres_image = cv2.imread(pres_img_path)
    # print(pres_json_file.replace("json", "jpg"))
    # print(pres_img_path)

    print(40*"+")
    for obj_bbox in pres_data:
        id = obj_bbox["id"]
        text = obj_bbox["text"]
        label = obj_bbox["label"]
        bbox = obj_bbox["box"] 
        mapping = ""
        if(label =="drugname"):
            mapping = obj_bbox["mapping"]
        if label == "drugname":
            print("label: ", text)

        # Key information extraction by rule
        if len(text) > 7:
            prob = difflib.SequenceMatcher(None, text[:10], "Chuẩn đoán").ratio()
            print(f'text[:10]: {text[:9]}, prob: {prob}')
            if prob > 0.7:
                diagnoses = text
                pres_image = cv2.rectangle(pres_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 1)
                pres_image = write_text(pres_image, "diagnose: "+text, (bbox[0], bbox[1]))
            else:
                drugnames_predict = difflib.get_close_matches(text, AllDrugnames)
                probs = []
                for name in drugnames_predict:
                    prob = difflib.SequenceMatcher(None, text, name).ratio()
                    probs.append(prob)
                if drugnames_predict != []:
                    drugname = drugnames_predict[np.argmax(probs)]
                    # print("drugnames_predict: ", drugnames_predict)
                    # print("label predict: ", drugname)
                    pres_image = cv2.rectangle(pres_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 255), 1)
                    pres_image = write_text(pres_image, "drugname: "+text, (bbox[0], bbox[1]))

    cv2.imwrite(dst+pres_json_file.replace("json", "png"), pres_image)

#     cv2.imshow("pres", pres_image)
#     if cv2.waitKey(0) == ord('q'):
#         break
# cv2.destroyAllWindows()